#### Importing the Libraries

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import csv
import tkinter as tk
from tkinter import filedialog

root = tk.Tk()
root.withdraw()


#### Defining a few useful functions #####


def get_data_from_csv(filepath,Time_data=False):#gets capacitance and frequency data from the csv file
    
    csv_fixer(filepath)
    
    new_file_name = filepath.with_name(filepath.stem+"_fixed.csv")
    
    file = pd.read_csv(new_file_name,index_col = 0,header = 2,sep = '\t')
        
    Cap_list = []#list of capacitance data for each loop
    Freq_list = []#list of frequency data for each loop
    Time_list = []#list of time data for each loop
    
    TIME = file["Point Number"]
    Sweep = file["Result Number"]
    Cap = file["Impedance Imaginary (Ohms)"]#for some reason all collumns are shifted by one
    Freq = file["Time"]                 #so pandas places the capacitance data on the "Impedance Imaginary (Ohms)" collumn
                                            #and the frequency data on the "Time" collumn
    
    number_of_sweeps = int(Sweep[len(Sweep)-1])
    
    for l in range(number_of_sweeps):
        Cap_list.append(Cap[Sweep == l+1])
        Freq_list.append(Freq[Sweep == l+1])
        Time_list.append(TIME[Sweep == l+1])
        
    os.remove(new_file_name)#deletes the "fixed" file after it's done
    
    if(Time_data):
        return Cap_list,Freq_list,Time_list,number_of_sweeps
    
    return Cap_list,Freq_list,number_of_sweeps



def csv_fixer(filepath,skip_lines=3):#shapes the data from SMART's output into something that can be read using pandas
                            #i.e. changes the decimal marker from "," to "." and separates the names of collumns using tab (\t)
                            #then saves it in a file called "(file_name)_fixed.csv"
    
    text = []
    
    with open(filepath) as fp:# reads the input file and copies it to the 'text' variable
        
        for counter in range(skip_lines):#these first few lines are only to ensure the file outputted is the same 
            line = fp.readline()  #as the original with the exception of our few wanted changes, 
            text.append(line)       #even though these few lines are useless and will be ignored later.
            counter += 1
   

        line = fp.readline() #changes the collumn name separators
        text.append(line.replace(',','\t'))
        
        line = fp.readline()
        
        while line:
            
            text.append(line.replace(',','.'))#replaces the decimal marker
            
            line = fp.readline()

            
    new_file_name = filepath.with_name(filepath.stem+"_fixed.csv")
    
    
    with open(new_file_name,'w') as f:#creates the new 'fixed' file
    
        for phrase in text:
            f.write(phrase)
            

def time_fixer(wrong_time):#takes the time from smarts output and converts it to minutes
    
    time_split = wrong_time.split(":")
                
    corrected_time = float(time_split[3])/60 + float(time_split[2]) + float(time_split[1])*60 + float(time_split[0])*60*24
    
    return corrected_time

    
def list_creator(start=1,end=1,step=1): #a more intuitive version of np.arange
    return np.arange(start-1,end,step) #the default return is np.array([0])
    

#### THIS BLOCK HANDLES THE FILE ACQUISITION ####

In [ ]:
#if all files are on the same folder you can just keep number_of_files as 0 and use multi-selection on the pop up window
#if not you have to manually set how many files you are going to open

number_of_files = 1

data_files = []

    
for i in range(number_of_files):#opens the files and gets their addresses
        
    data_files_pre = list(filedialog.askopenfilenames())

    for x in data_files_pre:
        data_files.append(Path(x))
        
    if(len(data_files) >= number_of_files):
        break 

number_of_files = len(data_files)



## THIS USED TO BE USEFUL... NOW I JUST DONT WANNA THROW IT AWAY ><   
    
#default_folder = "Monograph_Script_Images"

#if(not os.path.exists(default_folder)):
#    os.makedirs(default_folder)

    
#save_folder = Path(os.getcwd()+"/"+default_folder)

#### THIS SETS CONDITIONS TO EXCLUDE SPECIFIC SWEEPS FROM SPECIFIC FILES 

In [ ]:
def sweep_list(total_number_of_sweeps=0,file_number=0):#this function returns a list with the sweeps you want for 
                                       #each file_number and should be edited when needed
    
    file_number += 1 #accounts for file_number starting at 0 and going to total_number_of_files - 1
    
    ####it is easier if you use the functions list_creator and np.concatenate for this ####
    
    ###### code starts here ####
    
    #if(file_number == 3):
    #    return list_creator(40,total_number_of_sweeps,2)
    
    return np.concatenate((list_creator(1,1,1),list_creator(10,50,10)))
    
    #return np.arange(0,180)

#### THIS IS WHERE YOU CHANGE THE LABELS FOR EACH CURVE ON YOUR PLOT

In [ ]:
def condition_labelling(sweep=0,file_number=0):# sets the label of a curve according to the sweep number and the file number
                                        #returns a string and should be edited when needed.  
    
    sweep += 1 #accounts for sweep starting at 0 and going to total_number_of_sweeps - 1 and same for file_number
    file_number += 1
    
    #if(file_number == 1):
    #    return "IDE1 Bare"
    #elif(file_number == 2):
    #    return "IDE1 Filme"
    
    return ("Varredura "+str(sweep))

##### AFTER THIS LINE YOU SHOULD ONLY CHANGE PLOT SETTINGS SUCH AS X/Y SCALES AND LIMITS ####

In [ ]:
#defining a few parameters here

img_size = 150 #dpi
font_size = 13
font_axis_ticks = 12
number_of_label_columns = 2
meio = "agua" #ar or agua

## plotting happens here

for file_number in range(number_of_files):

    file = data_files[file_number]#this file_number can be used to name the electrodes later, so if in each file 
                                #you have a different electrode just use this "file_number" variable to set the label properly
    

    
    Capacitance, Frequency, number_of_sweeps = get_data_from_csv(file)
    

    
    if(meio == "ar"):

        for sweep in sweep_list(number_of_sweeps,file_number):
            
            plt.plot(
                    Frequency[sweep],
                     (10**12)*Capacitance[sweep],
                     "-o",
                     markersize = 3,
                     label = condition_labelling(sweep,file_number)
                     )
                
            plt.xlabel("Frequência (Hz)",fontsize=font_size)
            plt.ylabel("Capacitância Real (pF)",fontsize=font_size)
            plt.legend(loc = "upper right",ncol = number_of_label_columns)
            plt.yticks(fontsize=font_axis_ticks)
            plt.xticks(fontsize=font_axis_ticks)
            plt.xlim([1,10**3])
            #plt.xlim([10**3,10**6])
            #plt.ylim([7,8.5])
            #plt.yscale("log")
            plt.xscale("log")
                
    elif(meio == "agua"):
        
        for sweep in sweep_list(number_of_sweeps,file_number):
            
            plt.plot(
                    Frequency[sweep],
                    Capacitance[sweep],
                    "-o",
                    markersize = 3,
                    label = condition_labelling(sweep,file_number)
                    )
                
            plt.xlabel("Frequência (Hz)",fontsize=font_size)
            plt.ylabel("Capacitância Real (F)",fontsize=font_size)
            plt.legend(loc = "upper right",ncol = number_of_label_columns)
            plt.yticks(fontsize=font_axis_ticks)
            plt.xticks(fontsize=font_axis_ticks)
            #plt.xlim([1,10**3])
            #plt.xlim([10**3,10**6])
            #plt.ylim([7,8.5])
            plt.yscale("log")
            plt.xscale("log")


#saving the figure

answer = str(input("save figure (y/n)?\n"))

if(answer == "yes" or answer == 'y' or answer == "sim" or answer == 's'):
    save_as = Path(filedialog.asksaveasfilename(filetypes = (("png files","*.png"),("jpeg files","*.jpg"),("all files","*.*"))).replace(" ","_"))
    plt.savefig(save_as,dpi=img_size)

plt.show()

#### open the folder where the image was saved or the image itself directly from jupyter

In [ ]:
path = save_as
#path = save_as.parent
path = os.path.realpath(path)

try:
    os.startfile(path)
except:
    os.startfile(path+".png")

#### NOW PLOTTING CAPACITANCE (ON A FIXED FREQUENCY) OVER TIME

In [ ]:
fixed_freq = 10**3

#### code starts here ####

CapkHz = []
TimkHz = []

Capacitance, Frequency,TIME, number_of_sweeps = get_data_from_csv(Path(filedialog.askopenfilename()),Time_data = True)

if(number_of_sweeps > 1):
    for sweep in range(number_of_sweeps):
        f = Frequency[sweep].values.tolist()#the price you pay for using pandas....
        idx = f.index(float(fixed_freq))
        cap = Capacitance[sweep].values.tolist()[idx]
        tim = TIME[sweep].values.tolist()[idx]
        CapkHz.append(cap)
        TimkHz.append(time_fixer(tim))

In [ ]:
for i in range(len(CapkHz)):#in case you want to use pF
    CapkHz[i] *= 10**12

In [ ]:
#plotting preferences go here

plt.plot(TimkHz,CapkHz,"-o",markersize = 3)
plt.yticks(fontsize=10)
plt.xticks(fontsize=10)
plt.ylabel("Capacitância Real em 1kHz (pF)",fontsize=12)
plt.xlabel("Tempo (Minutos)",fontsize=12)
#plt.yscale("log")
#plt.xscale("log")

#saving the figure

answer = str(input("save figure (y/n)?\n"))

if(answer == "yes" or answer == 'y' or answer == "sim" or answer == 's'):
    save_as = Path(filedialog.asksaveasfilename(filetypes = (("png files","*.png"),("jpeg files","*.jpg"),("all files","*.*"))).replace(" ","_"))
    plt.savefig(save_as,dpi=150)
    
plt.show()

In [ ]:
path = save_as
#path = save_as.parent
path = os.path.realpath(path)

try:
    os.startfile(path)
except:
    os.startfile(path+".png")